In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
import sklearn

In [2]:
tf.__version__

'2.17.0'


## Loading the dataset

In [3]:
df = pd.read_csv("Churn_Modelling.csv", index_col = 'RowNumber')
df.head()

,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
RowNumber,,,,,,,,,,,,,
1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [4]:
# Removing the unnecessary columns from the training dataset (x)
# we will be removing 'CustomerID', 'Surname' columns
x = df.iloc[:, 2:-1].values
y = df.iloc[:, -1].values

In [5]:
x

array([[619, 'France', 'Female', ..., 1, 1, 101348.88],
       [608, 'Spain', 'Female', ..., 0, 1, 112542.58],
       [502, 'France', 'Female', ..., 1, 0, 113931.57],
       ...,
       [709, 'France', 'Female', ..., 0, 1, 42085.58],
       [772, 'Germany', 'Male', ..., 1, 0, 92888.52],
       [792, 'France', 'Female', ..., 1, 0, 38190.78]], dtype=object)

In [6]:
y

array([1, 0, 1, ..., 1, 1, 0])

# Data Preprocessing

### Encoding the categorical data

In [7]:
# Label Encoding gender column
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
x[:, 2] = le.fit_transform(x[:, 2])

In [8]:
x

array([[619, 'France', 0, ..., 1, 1, 101348.88],
       [608, 'Spain', 0, ..., 0, 1, 112542.58],
       [502, 'France', 0, ..., 1, 0, 113931.57],
       ...,
       [709, 'France', 0, ..., 0, 1, 42085.58],
       [772, 'Germany', 1, ..., 1, 0, 92888.52],
       [792, 'France', 0, ..., 1, 0, 38190.78]], dtype=object)

### One hot encoding the 'Geography' Column

In [9]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
ct = ColumnTransformer(transformers=[('encoder', OneHotEncoder(), [1])], remainder='passthrough')
x = np.array(ct.fit_transform(x))

In [10]:
print(x)

[[1.0 0.0 0.0 ... 1 1 101348.88]
 [0.0 0.0 1.0 ... 0 1 112542.58]
 [1.0 0.0 0.0 ... 1 0 113931.57]
 ...
 [1.0 0.0 0.0 ... 0 1 42085.58]
 [0.0 1.0 0.0 ... 1 0 92888.52]
 [1.0 0.0 0.0 ... 1 0 38190.78]]


### Splitting the dataset into the Training set and Test set

In [11]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size = 0.2, random_state = 0)

### Feature Scaling

In [12]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

# Model Building

#### Initialising ANN

In [13]:
ann = tf.keras.models.Sequential()

In [14]:
## Adding input layer & first hidden layer

# A object of dense class is used to add a fully connected layer to ann- 
# activation -> relu - rectifier activation function
# units -> number of hidden neurons in first layer
ann.add(tf.keras.layers.Dense(units=6, input_dim=12 ,activation='relu'))

## Adding second hidden layer
ann.add(tf.keras.layers.Dense(units=12, activation='relu'))

## Adding output layer
#in classification where we have more than 2 classes, activation should be softmax and not sigmoid

ann.add(tf.keras.layers.Dense(units=1, activation='sigmoid'))


/Users/saumyagupta/Documents/ML Projects/Deep Learning/Customer_Churn_Prediction_ANN/.venv/lib/python3.12/site-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


#### Compiling the ANN

In [15]:
# optimiser - adam - performs the stochastic gradient descent
# loss - loss function - when we have a binary classification the loss must be 'binary_crossentropy' if there are more categories then loss must be 'categorical_crossentropy'
# metrics - metric to evaluate our ann's performance
ann.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])

#### Training the ANN

In [16]:
ann.fit(X_train, y_train, batch_size = 32, epochs = 100)

Epoch 1/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 355us/step - accuracy: 0.6242 - loss: 0.6409
Epoch 2/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 281us/step - accuracy: 0.8069 - loss: 0.4678
Epoch 3/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 275us/step - accuracy: 0.8063 - loss: 0.4367
Epoch 4/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 275us/step - accuracy: 0.8130 - loss: 0.4362
Epoch 5/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 276us/step - accuracy: 0.8191 - loss: 0.4180
Epoch 6/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 289us/step - accuracy: 0.8259 - loss: 0.4048
Epoch 7/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 298us/step - accuracy: 0.8316 - loss: 0.4021
Epoch 8/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 282us/step - accuracy: 0.8293 - loss: 0.3967
Epoch 9/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 280us/step - accuracy: 0.8403 - loss: 0.3760
Epoch 10/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 276us/step - accuracy: 0.8496 - loss: 0.3669
Epoch 11/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 274us/step - accuracy: 0.8524 - loss: 0.3591
Epoch 12/100
250/25

#### Making the prediction for a single observation

In [17]:
print(ann.predict(sc.transform([[1, 0, 0, 600, 1, 40, 3, 60000, 2, 1, 1, 50000]])) > 0.5)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
[[False]]


#### Making predictions for test set

In [18]:
y_pred = ann.predict(X_test)
y_pred = (y_pred > 0.5)
print(np.concatenate((y_pred.reshape(len(y_pred),1), y_test.reshape(len(y_test),1)),1))

63/63 ━━━━━━━━━━━━━━━━━━━━ 0s 251us/step
[[0 0]
 [0 1]
 [0 0]
 ...
 [0 0]
 [0 0]
 [0 0]]


#### Making the confusion matrix

In [19]:
from sklearn.metrics import confusion_matrix, accuracy_score
cm = confusion_matrix(y_test, y_pred)
print(cm)
accuracy_score(y_test, y_pred)

[[1511   84]
 [ 192  213]]


0.862